In [2]:
%matplotlib notebook
import os
import os.path as path
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Combobox, interact
import ipywidgets as widgets
from IPython.display import display

# Local module and scripts
from pyccapt.calibration.calibration_tools import tools, data_tools, variables, widgets
from pyccapt.calibration.mc import mc_tools
# The initial value for flight path length, which is the distance of local electrod to detector 
# flightPathLength = 110 # mm 
# # The initial value for t_0
# t0 = 51.74 # ns

flightPathLength = 243 # 105.35 # mm 
# The initial value for t_0
t0 = 33 #68.48 # ns

analysis = False
save = True

In [ ]:
# dataset name
# AL_data_b
# OLO_AL_6_data
# OLO_W_6_data
# OLO_Ni_8_data
# X6Cr17_2V30Min_5_data
# data_1029_Jun-27-2022_15-53_Pd
# data_115_Jul-27-2022_17-44_Powersweep3

In [3]:
tdc, dataset = widgets.dataset_tdc_selection()
display(tdc, dataset)

Dropdown(description='TDC model:', options=('surface_concept', 'roentdec'), value='surface_concept')

Text(value='OLO_AL_6_data', description='Dataset:', placeholder='Paste ticket description here!')

$$\textbf{You can specify which dataset to use in below block}$$

In [7]:
tdc_model = tdc.value

dataset_name = dataset.value

p = path.abspath(path.join("", "../../../.."))

variables.init()
variables.path = os.path.join(p, 'tests//results//load_crop')
variables.result_path = os.path.join(p, 'tests/results/physics_experiment/' + dataset_name)
if not os.path.isdir(variables.result_path):
        os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
        
filename = variables.path + '//' + dataset_name + '//' + dataset_name + '_cropped' + '.h5'



head, tail = os.path.split(filename)
figname = os.path.splitext(tail)[0]

data = data_tools.read_hdf5_through_pandas(filename)


dld_highVoltage = data['dld/high_voltage'].to_numpy()
dld_laserIntensity = data['dld/laser_intensity'].to_numpy()
dld_t = data['dld/t'].to_numpy()
dld_x = data['dld/x'].to_numpy()
dld_y = data['dld/y'].to_numpy()

In [ ]:
dataset_name_base = 'data_101_Jul-27-2022_16-45_test.h5'

filename = variables.path + '//' + dataset_name + '//' + dataset_name + '_cropped' + '.h5'



head, tail = os.path.split(filename)
figname = os.path.splitext(tail)[0]

data_b = data_tools.read_hdf5_through_pandas(filename)


dld_highVoltage_b = data['dld/high_voltage'].to_numpy()
dld_laserIntensity_b = data['dld/laser_intensity'].to_numpy()
dld_t_b = data['dld/t'].to_numpy()
dld_x_b = data['dld/x'].to_numpy()
dld_y_b = data['dld/y'].to_numpy()

In [ ]:
data

In [ ]:
# t0
dld_t = dld_t - t0

In [ ]:
# Remove negative and zero TOF
threshold = 70
dld_highVoltage = dld_highVoltage[dld_t>threshold]
dld_laserIntensity = dld_laserIntensity[dld_t>threshold]
dld_x = dld_x[dld_t>threshold]
dld_y = dld_y[dld_t>threshold]
dld_t = dld_t[dld_t>threshold]

In [ ]:
fig1, ax1 = plt.subplots(figsize=(8, 4))

yaxis = dld_t[dld_t < 5000]  # dld_t
# y[y>5000] = 0
xaxis = np.arange(len(yaxis))
heatmap, xedges, yedges = np.histogram2d(xaxis, yaxis, bins=(1200, 800))
heatmap[heatmap == 0] = 1  # to have zero after apply log
heatmap = np.log(heatmap)
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
# set x-axis label
ax1.set_xlabel("hit sequence number", color="red", fontsize=14)
# set y-axis label
ax1.set_ylabel("time of flight [ns]", color="red", fontsize=14)
plt.title("Experiment history")
plt.imshow(heatmap.T, extent=extent, origin='lower', aspect="auto")

# plot high voltage curve
ax2 = ax1.twinx()

# dldGroupStorage[0] --> dld/high_voltage
high_voltage = dld_highVoltage[dld_t < 5000]
xaxis = np.arange(len(high_voltage))
ax2.plot(xaxis, high_voltage, color='r', linewidth=2)
ax2.set_ylabel("DC voltage [V]", color="blue", fontsize=14)
plt.show()

$$\textbf{Below plotted graph offers peak selection. You can select relevant peaks you want for computation.}$$

In [ ]:
mc_seb_ini = mc_tools.tof2mc(dld_t, 0, dld_highVoltage, dld_highVoltage, dld_x, dld_y, flightPathLength, mode='laser_pulse')
max_hist_ini, left_right_peaks_ini, peaks_sides_ini, max_paek_edges_ini, index_max_ini = tools.massSpecPlot(mc_seb_ini[mc_seb_ini < 300], 0.1, prominence=50, distance=100, text_loc='right', percent=50, plot=True, fig_name=figname)
mrp = (max_hist_ini / (left_right_peaks_ini[1] - left_right_peaks_ini[0]))
print('Mass resolving power for the highest peak (MRP --> m/m_2-m_1):', mrp)
for i in range(len(peaks_sides_ini)):
    print('Peaks ', i, 'is: {:.2f}'.format(peaks_sides_ini[i,0]), 'peak window sides are: {:.2f} - {:.2f}'.format(peaks_sides_ini[i,2], peaks_sides_ini[i,3]))

In [ ]:
# OD = (dld_laserIntensity - 265) / (270*2)
# dld_laserIntensity_p = 10.0**OD

dld_laserIntensity_p = dld_laserIntensity / np.max(dld_laserIntensity)

high_voltage = (dld_highVoltage - 6720) / 6720

fig1, ax1 = plt.subplots(figsize=(8, 4))
x = plt.scatter(dld_laserIntensity_p, high_voltage, color="blue", label='points', alpha=0.1)
ax1.set_ylabel("HV (Laser enhancement)", color="red", fontsize=10)
ax1.set_xlabel("Laser intensity (Fraction of laser power)", color="red", fontsize=10)

p = np.polyfit(dld_laserIntensity_p, high_voltage, 2)

y = plt.plot(dld_laserIntensity_p, np.polyval(p, dld_laserIntensity_p), color='red',label='Best Fit-2')


plt.legend(handles=[x, y[0]], loc='upper left')
if save==True:
    plt.savefig(variables.result_path + "//graph_%s.svg" % figname, format="svg", dpi=600)
    plt.savefig(variables.result_path + "//graph_%s.png" % figname, format="png", dpi=600)
plt.show()